In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
class DiGraph:
    def __init__(self) -> None:
        self.max_v = -1
        self.edges = []

    def add_edge(self, u: int, v: int, x: float) -> None:
        self.max_v = max(self.max_v, max(u, v))
        self.edges.append((u, v, x))
        
    def get_n(self) -> int:
        return self.max_v + 1

    def get_m(self) -> int:
        return len(self.edges)

    def to_array(self) -> np.array:
        g = np.zeros((self.max_v + 1, self.max_v + 1), np.float32)
        for edge in self.edges:
            g[edge[0]][edge[1]] = 1 + edge[2]
        return g

In [3]:
def read_ego_net(ego_net_path):
    cur_ego_id = -1
    cur_ego_net = None
    cur_ego_net_x1 = None
    cur_ego_net_x2 = None
    cur_ego_net_x3 = None
    cur_ego_net_time = None
    with open(ego_net_path, 'r') as ego_net_f:
        ego_net_f.readline()
        for line in ego_net_f:
            line = line.split(',')
            ego_id, u, v = int(line[0]), int(line[1]), int(line[2])
            x2, x3 = float(line[5]), float(line[6])

            if ego_id != cur_ego_id:
                if cur_ego_id != -1:
                    yield cur_ego_id, cur_ego_net, cur_ego_net_x1, cur_ego_net_x2, cur_ego_net_x3, cur_ego_net_time
                assert cur_ego_id < ego_id
                cur_ego_id = ego_id
                cur_ego_net = DiGraph()
                cur_ego_net_x1 = DiGraph()
                cur_ego_net_x2 = DiGraph()
                cur_ego_net_x3 = DiGraph()
                cur_ego_net_time = DiGraph()
            
            cur_ego_net.add_edge(u, v, 0)
            if line[4] == "":
                cur_ego_net_x1.add_edge(u, v, -1)
            else:
                cur_ego_net_x1.add_edge(u, v, float(line[4]))
            cur_ego_net_x2.add_edge(u, v, x2)
            cur_ego_net_x3.add_edge(u, v, x3)
            if line[3] == "":
                cur_ego_net_time.add_edge(u, v, -1)
            else:
                cur_ego_net_time.add_edge(u, v, float(line[3]))

        if cur_ego_id != -1:
            yield cur_ego_id, cur_ego_net, cur_ego_net_x1, cur_ego_net_x2, cur_ego_net_x3, cur_ego_net_time

In [4]:
train = pd.read_csv(
    './data/train.csv', index_col='ego_id', usecols=['ego_id', 'u', 'v', 'x1'], 
    dtype={'ego_id': 'int64', 'u': 'int32', 'v': 'int32', 'x1': 'float32'}
)
train

,u,v,x1
ego_id,,,
0,131,84,5.669200e-07
0,135,164,6.246274e-02
0,47,15,0.000000e+00
0,5,4,4.962974e-02
0,176,219,1.237935e+00
...,...,...,...
1709396984692,3,5,2.307750e+00
1709396984692,1,5,3.729143e+00
1709396984692,1,7,4.286984e+00


In [5]:
test = pd.read_csv(
    './data/test.csv', index_col='ego_id', usecols=['ego_id', 'u', 'v', 'x1'], 
    dtype={'ego_id': 'int64', 'u': 'int32', 'v': 'int32', 'x1': 'float32'}
)
test

,u,v,x1
ego_id,,,
8,20,19,3.839089e-04
8,131,125,4.034464e-01
8,73,56,8.554643e-05
8,0,4,2.886418e-01
8,63,73,4.281692e-07
...,...,...,...
1709396984676,89,0,1.167843e+00
1709396984676,84,87,1.179100e-06
1709396984676,8,18,1.175182e+00


In [6]:
def build_X(X, ego_net, ego_net_x1, ego_net_x2, ego_net_x3, ego_net_time):
    vertex_cnt = ego_net_x2.get_n()
    
    vertex_cnt = ego_net.get_n()
    edge_cnt = ego_net.get_m()
    
    X['vertex_cnt'] = vertex_cnt
    X['edge_cnt'] = edge_cnt
    X['edge-vertex_cnt'] = edge_cnt - vertex_cnt
    X['density'] = 2 * edge_cnt / vertex_cnt / (vertex_cnt - 1)
    
    g = ego_net.to_array()
    degree = g.sum(axis=1)
    
    X['u_neighbour_cnt'] = degree[X.u]
    X['v_neighbour_cnt'] = degree[X.v]
    
    sc = g.dot(g.T).flatten()
    X['SC'] = sc[X.u * vertex_cnt + X.v]
    
    degree = 1 + degree.reshape((-1, 1))
        
    aa = (g / degree).dot(g.T).flatten()
    X['AA'] = aa[X.u * vertex_cnt + X.v]
    aa = (g / (1 + np.sqrt(degree))).dot(g.T).flatten()
    X['AA_sqrt'] = aa[X.u * vertex_cnt + X.v]
    aa = (g / (1 + np.log(degree))).dot(g.T).flatten()
    X['AA_log'] = aa[X.u * vertex_cnt + X.v]
        
    g = ego_net_x1.to_array()
    
    X['r_x1'] = g[X.v, X.u]
    
    g = ego_net_time.to_array()
    
    X['t'] = g[X.u, X.v]
    X['r_t'] = g[X.v, X.u]
    
    X['u_time_0'] = g[X.u, 0]
    X['v_time_0'] = g[X.v, 0]
    
    inv = g.copy()
    inv[inv > 0] = 1 / inv[inv > 0]
    inv_sum = inv.sum(axis=1)
    X['u_t_sum'] = inv_sum[X.u]
    X['v_t_sum'] = inv_sum[X.v]
    
    t_min = g.copy()
    t_min[t_min == 0] = t_min.max()
    X['min_time'] = t_min.min()
    t_min = t_min.min(axis=1)
    X['u_t_min'] = t_min[X.u]
    X['v_t_min'] = t_min[X.v]
    
    sc = inv.dot(inv.T).flatten()
    X['SC_time'] = sc[X.u * vertex_cnt + X.v]
    
    g = ego_net_x2.to_array()
    
    X['x2'] = g[X.u, X.v]
    X['r_x2'] = g[X.v, X.u]
    
    x2_sum = g.sum(axis=1)
    X['u_x2_sum'] = x2_sum[X.u]
    X['v_x2_sum'] = x2_sum[X.v]
    
    x2_max = g.max(axis=1)
    X['u_x2_max'] = x2_max[X.u]
    X['v_x2_max'] = x2_max[X.v]
    
    sc = g.dot(g.T).flatten()
    X['SC_x2'] = sc[X.u * vertex_cnt + X.v]
    
    g = ego_net_x3.to_array()
    
    X['x3'] = g[X.u, X.v]
    X['r_x3'] = g[X.v, X.u]
    
    x3_sum = g.sum(axis=1)
    X['u_x3_sum'] = x3_sum[X.u]
    X['v_x3_sum'] = x3_sum[X.v]
    
    x3_max = g.max(axis=1)
    X['u_x3_max'] = x3_max[X.u]
    X['v_x3_max'] = x3_max[X.v]
    
    sc = g.dot(g.T).flatten()
    X['SC_x3'] = sc[X.u * vertex_cnt + X.v]

In [7]:
train_df = []
for ego_id, ego_net, ego_net_x1, ego_net_x2, ego_net_x3, ego_net_time in \
tqdm(read_ego_net('./data/train.csv'), total=61786):
    X = train.loc[ego_id].reset_index()
    build_X(X, ego_net, ego_net_x1, ego_net_x2, ego_net_x3, ego_net_time)
    train_df.append(X)

  0%|          | 0/61786 [00:00<?, ?it/s]

In [8]:
train_df = pd.concat(train_df)
train_df

,ego_id,u,v,x1,vertex_cnt,edge_cnt,edge-vertex_cnt,density,u_neighbour_cnt,v_neighbour_cnt,...,u_x2_max,v_x2_max,SC_x2,x3,r_x3,u_x3_sum,v_x3_sum,u_x3_max,v_x3_max,SC_x3
0,0,131,84,5.669200e-07,300,1642,1342,0.036611,7.0,10.0,...,1.000000,1.000000,2.000000,1.0,1.0,7.0,10.0,1.0,1.0,2.0
1,0,135,164,6.246274e-02,300,1642,1342,0.036611,4.0,5.0,...,1.000000,1.000000,2.000000,1.0,0.0,4.0,5.0,1.0,1.0,2.0
2,0,47,15,0.000000e+00,300,1642,1342,0.036611,3.0,3.0,...,1.000000,1.000000,0.000000,2.0,0.0,6.0,3.0,2.0,1.0,0.0
3,0,5,4,4.962974e-02,300,1642,1342,0.036611,24.0,10.0,...,1.693147,1.000000,7.693147,1.0,1.0,24.0,10.0,1.0,1.0,7.0
4,0,176,219,1.237935e+00,300,1642,1342,0.036611,15.0,18.0,...,3.995732,6.860786,11.860786,1.0,0.0,19.0,24.0,2.0,2.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,0,253,158,1.497307e-01,300,1642,1342,0.036611,4.0,4.0,...,1.000000,1.000000,1.000000,2.0,1.0,7.0,4.0,2.0,1.0,1.0
1638,0,259,244,1.827714e+00,300,1642,1342,0.036611,10.0,11.0,...,6.897154,3.833213,5.944439,2.0,0.0,15.0,15.0,2.0,2.0,7.0
1639,0,209,127,6.636844e-02,300,1642,1342,0.036611,19.0,21.0,...,5.454347,3.890372,8.609438,1.0,1.0,19.0,34.0,1.0,2.0,13.0
1640,0,187,241,3.756446e+00,300,1642,1342,0.036611,10.0,18.0,...,6.961005,4.295837,11.289363,1.0,0.0,10.0,18.0,1.0,1.0,4.0


In [9]:
test_df = []
for ego_id, ego_net, ego_net_x1, ego_net_x2, ego_net_x3, ego_net_time in\
tqdm(read_ego_net('./data/test.csv'), total=20596):
    X = test.loc[ego_id].reset_index()
    build_X(X, ego_net, ego_net_x1, ego_net_x2, ego_net_x3, ego_net_time)
    test_df.append(X)

  0%|          | 0/20596 [00:00<?, ?it/s]

In [10]:
test_df = pd.concat(test_df)
test_df

,ego_id,u,v,x1,vertex_cnt,edge_cnt,edge-vertex_cnt,density,u_neighbour_cnt,v_neighbour_cnt,...,u_x2_max,v_x2_max,SC_x2,x3,r_x3,u_x3_sum,v_x3_sum,u_x3_max,v_x3_max,SC_x3
0,8,20,19,3.839089e-04,194,1018,824,0.054377,12.0,7.0,...,1.000000,1.000000,4.000000,1.0,1.0,12.0,7.0,1.0,1.0,4.0
1,8,131,125,4.034464e-01,194,1018,824,0.054377,6.0,3.0,...,1.693147,2.098612,4.553259,1.0,0.0,6.0,3.0,1.0,1.0,2.0
2,8,73,56,8.554643e-05,194,1018,824,0.054377,10.0,12.0,...,1.000000,1.000000,4.000000,1.0,1.0,11.0,12.0,2.0,1.0,5.0
3,8,0,4,2.886418e-01,194,1018,824,0.054377,122.0,6.0,...,3.564949,1.000000,0.000000,1.0,0.0,122.0,6.0,1.0,1.0,0.0
4,8,63,73,4.281692e-07,194,1018,824,0.054377,8.0,10.0,...,1.000000,1.000000,3.000000,1.0,1.0,10.0,11.0,2.0,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,8,132,17,1.826740e+00,194,1018,824,0.054377,2.0,20.0,...,2.791759,6.493062,0.000000,1.0,0.0,2.0,20.0,1.0,1.0,0.0
1014,8,29,14,NaN,194,1018,824,0.054377,6.0,10.0,...,1.000000,1.000000,3.000000,1.0,0.0,6.0,10.0,1.0,1.0,3.0
1015,8,56,59,NaN,194,1018,824,0.054377,12.0,10.0,...,1.000000,2.386294,6.386294,1.0,1.0,12.0,10.0,1.0,1.0,5.0
1016,8,14,11,NaN,194,1018,824,0.054377,10.0,10.0,...,1.000000,1.000000,5.000000,1.0,0.0,10.0,10.0,1.0,1.0,5.0


In [11]:
val_df = test_df[test_df.x1.notna()].copy()
val_df

,ego_id,u,v,x1,vertex_cnt,edge_cnt,edge-vertex_cnt,density,u_neighbour_cnt,v_neighbour_cnt,...,u_x2_max,v_x2_max,SC_x2,x3,r_x3,u_x3_sum,v_x3_sum,u_x3_max,v_x3_max,SC_x3
0,8,20,19,3.839089e-04,194,1018,824,0.054377,12.0,7.0,...,1.000000,1.000000,4.000000,1.0,1.0,12.0,7.0,1.0,1.0,4.0
1,8,131,125,4.034464e-01,194,1018,824,0.054377,6.0,3.0,...,1.693147,2.098612,4.553259,1.0,0.0,6.0,3.0,1.0,1.0,2.0
2,8,73,56,8.554643e-05,194,1018,824,0.054377,10.0,12.0,...,1.000000,1.000000,4.000000,1.0,1.0,11.0,12.0,2.0,1.0,5.0
3,8,0,4,2.886418e-01,194,1018,824,0.054377,122.0,6.0,...,3.564949,1.000000,0.000000,1.0,0.0,122.0,6.0,1.0,1.0,0.0
4,8,63,73,4.281692e-07,194,1018,824,0.054377,8.0,10.0,...,1.000000,1.000000,3.000000,1.0,1.0,10.0,11.0,2.0,2.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,8,140,113,6.380812e-06,194,1018,824,0.054377,3.0,3.0,...,1.000000,1.000000,0.000000,1.0,1.0,3.0,3.0,1.0,1.0,0.0
1010,8,104,91,5.176676e-01,194,1018,824,0.054377,3.0,6.0,...,1.000000,1.000000,1.000000,1.0,1.0,3.0,6.0,1.0,1.0,1.0
1011,8,121,41,2.224251e-02,194,1018,824,0.054377,7.0,10.0,...,1.000000,1.693147,1.000000,1.0,1.0,7.0,12.0,1.0,2.0,2.0
1012,8,91,13,6.132724e-15,194,1018,824,0.054377,6.0,12.0,...,1.000000,1.000000,0.000000,1.0,1.0,6.0,12.0,1.0,1.0,0.0


In [12]:
sub = pd.read_csv('./data/submission.csv')
sub_values = set()
for ego_id, u, v in zip(sub.ego_id, sub.u, sub.v):
    sub_values.add((ego_id, u, v))

In [13]:
mask = []
for ego_id, u, v in zip(test_df.ego_id, test_df.u, test_df.v):
    mask.append((ego_id, u, v) in sub_values)

In [14]:
test_df = test_df[mask].drop_duplicates(subset=['ego_id', 'u', 'v']).copy()
test_df

,ego_id,u,v,x1,vertex_cnt,edge_cnt,edge-vertex_cnt,density,u_neighbour_cnt,v_neighbour_cnt,...,u_x2_max,v_x2_max,SC_x2,x3,r_x3,u_x3_sum,v_x3_sum,u_x3_max,v_x3_max,SC_x3
120,8,7,16,NaN,194,1018,824,0.054377,12.0,8.0,...,1.000000,1.000000,3.000000,1.0,1.0,12.0,8.0,1.0,1.0,3.0
141,8,8,29,NaN,194,1018,824,0.054377,13.0,6.0,...,1.000000,1.000000,5.000000,1.0,1.0,13.0,6.0,1.0,1.0,5.0
142,8,75,0,NaN,194,1018,824,0.054377,6.0,122.0,...,1.693147,3.564949,3.693147,1.0,1.0,6.0,122.0,1.0,1.0,3.0
183,8,0,151,NaN,194,1018,824,0.054377,122.0,5.0,...,3.564949,5.234107,3.609438,1.0,1.0,122.0,5.0,1.0,1.0,2.0
297,8,8,20,NaN,194,1018,824,0.054377,13.0,12.0,...,1.000000,1.000000,4.000000,1.0,0.0,13.0,12.0,1.0,1.0,4.0
302,8,152,65,NaN,194,1018,824,0.054377,15.0,16.0,...,1.000000,3.564949,10.025352,1.0,1.0,16.0,16.0,2.0,1.0,7.0
317,8,5,4,NaN,194,1018,824,0.054377,9.0,6.0,...,1.000000,1.000000,3.000000,1.0,1.0,9.0,6.0,1.0,1.0,3.0
347,8,55,0,NaN,194,1018,824,0.054377,11.0,122.0,...,1.000000,3.564949,7.000000,1.0,0.0,15.0,122.0,2.0,1.0,9.0
456,8,46,110,NaN,194,1018,824,0.054377,6.0,4.0,...,1.000000,1.000000,1.000000,1.0,0.0,6.0,7.0,1.0,2.0,2.0
461,8,74,159,NaN,194,1018,824,0.054377,13.0,3.0,...,1.000000,2.098612,0.000000,1.0,2.0,13.0,4.0,1.0,2.0,0.0


In [15]:
del train, test

In [16]:
import gc
gc.collect()

0

In [17]:
train_df['x1'].mean(), val_df['x1'].mean()

(0.666723, 0.43647254)

In [18]:
from catboost import CatBoostRegressor, Pool

train_pool = Pool(
    data=train_df.drop(['ego_id', 'u', 'v', 'x1'], axis=1),
    label=train_df['x1']
)

val_pool = Pool(
    data=val_df.drop(['ego_id', 'u', 'v', 'x1'], axis=1),
    label=val_df['x1']
)

2023-05-20 20:18:10,999 [129312] WARNING  py.warnings:109: [JupyterRequire] C:\Users\Konder\AppData\Roaming\Python\Python38\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index



In [19]:
params = {
    'task_type': 'CPU',
    'loss_function': 'RMSE',
    'iterations': 300,
    'max_depth': 5,
}

In [20]:
model_cb = CatBoostRegressor(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, plot=True, verbose=False, use_best_model=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [21]:
model_cb.get_feature_importance(prettified=True)

,Feature Id,Importances
0,x2,61.328028
1,r_x1,10.979588
2,t,4.163571
3,v_t_min,2.484091
4,v_neighbour_cnt,1.787329
5,r_t,1.716425
6,v_t_sum,1.508348
7,v_x2_max,1.412791
8,SC_x2,1.319794
9,u_time_0,1.259190


In [22]:
1 - np.min(model_cb.evals_result_['validation']['RMSE'])

0.3486398946476873

In [23]:
x1_pred = model_cb.predict(test_df)
x1_pred[x1_pred<0] = 0
x1_pred[x1_pred>50] = 50
test_df['x1'] = x1_pred

In [24]:
test_df = pd.merge(test_df[['ego_id', 'u', 'v', 'x1']], sub[['ego_id', 'u', 'v']], on = ['ego_id', 'u', 'v'], how='right')
test_df.to_csv('./x1,x2,x3,t_submit.csv', index=False)